<a href="https://colab.research.google.com/github/engineerJPark/FCN_Implementation/blob/main/FCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html 
https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
https://blog.promedius.ai/pytorch_dataloader_1/

- transform 처리
  - Resize는 복원이 non bilinear하게 할 방법을 찾을 것, 아니라면 그냥 whole size 이용
  - pooling 홀수도 잘 복원되게 하는 방법
- image 크기 안줄이고 1 batch로 처리
- dataset __getitem__() 수정
  - 기존 코드 삭제하고 transform함수만 사용할 것



In [57]:
import numpy as np
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from torchvision.models import resnet18, ResNet18_Weights

from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset


# for plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'

import collections
import PIL
import os


if torch.cuda.is_available():
  device = torch.device('cuda:0')
else:
  device = torch.device('cpu')

torch.manual_seed(1)
if device == 'cuda:0':
    torch.cuda.manual_seed_all(1)

resnet_pretrained = resnet18(weights=ResNet18_Weights.DEFAULT)
print('Done.')

Done.


In [58]:
from google.colab import drive
drive.mount('/content/gdrive')
ROOT_DIR = '/content/gdrive/MyDrive/voc_data'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [59]:
# sample_img = PIL.Image.open(os.path.join(ROOT_DIR, 'VOCdevkit/VOC2012', 'JPEGImages/2007_000032.jpg'))
# sample_img_np = np.array(sample_img)

# print(sample_img_np.shape)

# plt.figure(figsize=(8, 8))
# plt.imshow(sample_img)
# plt.show()

# plt.figure(figsize=(8, 8))
# plt.imshow(sample_img_np[:,:,0])
# plt.show()

# # sample_img_np = np.array(sample_img.convert('RGB'))

# plt.figure(figsize=(8, 8))
# plt.imshow(sample_img_np[:,:,2])
# plt.show()

In [60]:
# x = torch.randn((3, 281, 500), device=device ,dtype=torch.float)
# pool = nn.MaxPool2d(2, stride=2)
# out = pool(x)
# print(out.shape)

In [61]:
class VOCClassSegBase(torch.utils.data.Dataset):

    class_names = np.array([
        'background',
        'aeroplane',
        'bicycle',
        'bird',
        'boat',
        'bottle',
        'bus',
        'car',
        'cat',
        'chair',
        'cow',
        'diningtable',
        'dog',
        'horse',
        'motorbike',
        'person',
        'potted plant',
        'sheep',
        'sofa',
        'train',
        'tv/monitor',
    ])

    def __init__(self, root='/content/gdrive/MyDrive/voc_data', split='train', transform_tf=True):
        self.root = root
        self.split = split
        self.transform_tf = transform_tf
        self.transform = transforms.Compose([
            transforms.Resize((320,320), interpolation=InterpolationMode.NEAREST),
            transforms.Normalize(mean=(122.67891434, 116.66876762, 104.00698793), std=(255., 255., 255.))
        ])
        self.transform_lbl = transforms.Compose([
            transforms.Resize((320,320), interpolation=InterpolationMode.NEAREST)
        ])        

        dataset_dir = os.path.join(self.root, 'VOCdevkit/VOC2012')
        self.files = collections.defaultdict(list)
        for split in ['train', 'val']:
            imgsets_file = os.path.join(
                dataset_dir, 'ImageSets/Segmentation/%s.txt' % split)
            for did in open(imgsets_file):
                did = did.strip()
                img_file = os.path.join(dataset_dir, 'JPEGImages/%s.jpg' % did)
                lbl_file = os.path.join(
                    dataset_dir, 'SegmentationClass/%s.png' % did)
                self.files[split].append({
                    'img': img_file,
                    'lbl': lbl_file,
                })

    def __len__(self):
        return len(self.files[self.split])

    def __getitem__(self, index):
        # data file
        data_file = self.files[self.split][index]
        
        # load
        img_file = data_file['img']
        img = PIL.Image.open(img_file)
        img = torch.from_numpy(np.array(img)).float()

        lbl_file = data_file['lbl']
        lbl = PIL.Image.open(lbl_file)
        lbl = torch.from_numpy(np.array(lbl)).long()
        lbl = torch.unsqueeze(lbl, dim=0)

        # image preprocessing
        img = img.permute(2, 0, 1) # HWC -> CHW
        lbl[lbl == 255] = -1
        
        # print('img shape', img.shape)
        # print('lbl shape', lbl.shape)
        if self.transform_tf == True:
            return self.transform(img), self.transform_lbl(lbl) # lbl 부분이 문제
            # return self.transform(img), lbl
        else:
            return img, lbl


train_data = VOCClassSegBase(root=ROOT_DIR, split='train', transform_tf=True)

train_data_loader = DataLoader(dataset=train_data, batch_size = 1,
                               drop_last=True)

In [62]:
# Bilinear weights deconvolution Algorithm 알아보기
def bilinear_kernel_init(Cin, Cout, kernel_size):
  factor = (kernel_size + 1) // 2
  if kernel_size % 2 == 1:
    center = (kernel_size + 1) // 2 - 1
  else:
    center = (kernel_size + 1) // 2 - 0.5

  og = (torch.arange(kernel_size).reshape(-1,1), torch.arange(kernel_size).reshape(1,-1))
  filter = (1 - torch.abs(og[0] - center) / factor) * (1 - torch.abs(og[1] - center) / factor)

  weight = torch.zeros((Cin, Cout, kernel_size, kernel_size))
  weight[range(Cin), range(Cout), :, :] = filter
  return weight

In [63]:
class FCN18(nn.Module):
  def __init__(self, class_n): # class 20 + 1(bakcground)
    super().__init__()
    self.downsample1 = nn.Sequential( # 1/2 of the input
        resnet_pretrained.conv1,
        resnet_pretrained.bn1,
        resnet_pretrained.relu,
        resnet_pretrained.maxpool
    )
    self.downsample2 = resnet_pretrained.layer1 # 1/4 of the input
    self.downsample3 = resnet_pretrained.layer2 # 1/8 of the input, 여기서 1x1 거친것 하나 추출
    self.downsample4 = resnet_pretrained.layer3 # 1/16 of the input, 여기서 1x1 거친것 하나 추출
    self.downsample5 = resnet_pretrained.layer4 # 1/32 of the input, 여기서 1x1 거친것 하나 추출

    self.fully_conv_pool3 = nn.Conv2d(128, class_n, kernel_size=1) # get class score for each pixel
    nn.init.xavier_normal_(self.fully_conv_pool3.weight)

    self.fully_conv_pool4 = nn.Conv2d(256, class_n, kernel_size=1) # get class score for each pixel
    nn.init.xavier_normal_(self.fully_conv_pool4.weight)

    self.fully_conv_pool5 = nn.Conv2d(512, class_n, kernel_size=1) # get class score for each pixel
    nn.init.xavier_normal_(self.fully_conv_pool5.weight)
    
    # 보통 stride s, padding s/2, kernelsize 2s -> s배 만큼 이미지가 커진다.
    self.upsample_make_16s = nn.ConvTranspose2d(class_n, class_n, kernel_size=4, padding=1, stride=2, bias=False) # to 1/16
    self.upsample_make_16s.weight.data.copy_(bilinear_kernel_init(class_n, class_n, 4))

    self.upsample_make_8s = nn.ConvTranspose2d(class_n, class_n, kernel_size=4, padding=1, stride=2, bias=False) # to 1/8
    self.upsample_make_8s.weight.data.copy_(bilinear_kernel_init(class_n, class_n, 4))
    
    self.upsample_to_score = nn.ConvTranspose2d(class_n, class_n, kernel_size=16, padding=4, stride=8) # to 1
    self.upsample_to_score.weight.data.copy_(bilinear_kernel_init(class_n, class_n, 16))
    for param in self.upsample_to_score.parameters(): # freeze the last layer
      param.requires_grad = False


  def forward(self, x):
    x = self.downsample1(x)
    x = self.downsample2(x)
    x = self.downsample3(x)
    pool3_score = self.fully_conv_pool3(x)
    x = self.downsample4(x)
    pool4_score = self.fully_conv_pool4(x)
    x = self.downsample5(x)
    pool5_score = self.fully_conv_pool5(x)

    pool4_2x_conv7 = pool4_score + self.upsample_make_16s(pool5_score)
    pool3_2x_pool4_4x_conv7 = pool3_score + self.upsample_make_8s(pool4_2x_conv7)

    out = self.upsample_to_score(pool3_2x_pool4_4x_conv7)

    return out

In [64]:
epochs = 10
lr = 10e-4
weight_decay = 2e-4
momentum = 0.9
batch_size = 64

model = FCN18(3)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

In [70]:
# train code
def train(model, epochs, optimizer, criterion):
  for epoch in range(epochs):
    for train_img, train_gt_img in train_data_loader:

      print('train_img.shape:', train_img.shape)
      print('train_gt_img.shape:', train_gt_img.shape)
      N, C, H, W = train_img.shape
      N_t, C_t, H_t, W_t = train_gt_img.shape

      # print('train_img.shape:', N, C, H, W)
      # print('train_gt_img.shape:', train_gt_img.shape)
      # print(train_gt_img[0][0][150][150])
      
      train_img = train_img.to(device)
      train_gt_img = train_gt_img.squeeze(dim=1).to(device)

      print('train_gt_img.shape:', train_gt_img.shape)

      score_img = model(train_img.to(torch.float))
      score_img = score_img.permute(0,2,3,1).reshape(N*H*W, C)
      train_gt_img = train_gt_img.reshape(N_t*H_t*W_t, )

      print('score_img.shape:', score_img.shape)
      print('train_gt_img.shape:', train_gt_img.shape)

      loss = criterion(score_img, train_gt_img)

      optimizer.zero_grad()
      optimizer.step()
      loss.backward()

    print("epoch %d, loss : %f "%(epoch + 1, loss))
  print("Training End")

train(model, epochs, optimizer, criterion)

train_img.shape: torch.Size([1, 3, 320, 320])
train_gt_img.shape: torch.Size([1, 1, 320, 320])
train_gt_img.shape: torch.Size([1, 320, 320])
score_img.shape: torch.Size([102400, 3])
train_gt_img.shape: torch.Size([102400])


IndexError: ignored

In [ ]:
# segmentation : plot image

with open("/content/data/VOCdevkit/VOC2012/ImageSets/Segmentation/val.txt", 'r') as f:
  lines = f.readlines()
for i in range(len(lines)):
  lines[i] =  lines[i].strip('\n')
test_jpg = lines[0] + '.jpg'
test_image = PIL.Image.open(os.path.join(ROOT_DIR, 'VOCdevkit/VOC2012', "JPEGImages", test_jpg))

test_seg = model(test_image)

plt.figure(figsize=(8, 8))
plt.imshow(test_seg)
plt.show()

In [ ]:
# validation IoU



In [ ]:
# openset2voc 

